In [ ]:
class SecretAgent:
    # 类属性,应该在类的顶部
    # 在类属性前加_线,这意味着他不应该在类外被修改..这只是一种预定,并没有强制
    _codeword = ""

    def __init__(self, codename=""):
        # 初始化器,不同于构造函数
        # # 所有实例属性都应该是初始化器中

        self.codename = codename
        self._secret = []
        self.__format = "UTF-8"  # 为了防止被继承类修改

    # 实例方法
    def remember(self, secret):
        self._secret.append(secret)

    # 类方法
    @classmethod
    def setCodeword(cls, codeword):
        # 不用担心是实例还是类来调用类方法
        cls._codeword = codeword

    @classmethod
    def getCodeword(cls):
        return cls._codeword

    @classmethod
    def _encrypt(cls, msg, *, decrypt=False):
        code = sum(ord(c) for c in cls._codeword)
        if decrypt:
            code -= code
        return "".join(chr(ord(m) + code) for m in msg)

    @staticmethod
    def inquire(question=""):
        # 静态方法不会访问任何类属性和方法
        print("I dont know!")

    def _getsecret(self):
        return self._secret[-1] if self._secret else None

    def _setsecret(self, value):
        self._secret.append(self._encrypt(value))

    def _delsecret(self):
        self._secret = self._secret[:-1]  # 列表置空了

    # property是一种把方法当做属性的一种机制
    secret = property(fget=_getsecret, fset=_setsecret, fdel=_delsecret)

In [ ]:
SecretAgent._codeword = "parmesean"  # 如果在类属性中重新绑定或更改类属性,则所有都应用
mouse = SecretAgent("mouse")
weseal = SecretAgent("weseal")
print(mouse._codeword)
print(weseal._codeword)
mouse._codeword = "mouse"  # 可以通过实例访问类属性,这样会生成一个实例属性覆盖当前实例的类属性,其余的类属性不变
print(mouse._codeword)
print(weseal._codeword)
mouse.remember(("42,-72"))
print(mouse.getCodeword())
fox = SecretAgent("fox")
fox.setCodeword("The ducks quacks at midnight!")
print(mouse.getCodeword())  # The ducks quacks at midnight!
print(fox.getCodeword())  # The ducks quacks at midnight!
fox.inquire()


print(mouse.secret)
mouse.secret = "12345Mainstreet"
print(mouse.secret)
print(mouse._secret)
del mouse.secret
print(mouse._secret)

In [8]:
# 魔方方法,以两个下划线开头和结尾
import math


class GlobalCoordinates:
    def __init__(self, *, latitude, longtitude):
        self._lat_deg = latitude[0]
        self._lat_min = latitude[1]
        self._lat_sec = latitude[2]
        self._lat_dir = latitude[3]
        self._lon_deg = longtitude[0]
        self._lon_min = longtitude[1]
        self._lon_sec = longtitude[2]
        self._lon_dir = longtitude[3]

    @staticmethod
    def degree_from_decimal(dec, *, lat):
        if lat:
            direction = "S" if dec < 0 else "N"
        else:
            direction = "W" if dec < 0 else "E"
        dec = abs(dec)
        degrees = int(dec)
        dec -= degrees  # ?
        minutes = int(dec * 60)  # ?
        dec -= minutes / 60
        seconds = round(dec * 3600, 1)  # ?
        return (degrees, minutes, seconds, direction)

    @staticmethod
    def decimal_from_degrees(degrees, minutes, seconds, direction):
        dec = degrees + minutes / 60 + seconds / 3600
        if direction == "S" or direction == "W":
            dec = -dec
        return round(dec, 6)

    @property
    def latitude(self):
        return self.decimal_from_degrees(
            self._lat_deg, self._lat_min, self._lat_sec, self._lat_dir
        )

    @property
    def longtitude(self):
        return self.decimal_from_degrees(
            self._lon_deg, self._lon_min, self._lon_sec, self._lon_dir
        )

    def __str__(self):
        """主要用于调试,看对象的内容"""
        return (
            f"{self._lat_deg}°{self._lat_min}"
            f'{self._lat_sec}"{self._lat_dir} '
            f"{self._lon_deg}°{self._lon_min}"
            f'{self._lon_sec}"{self._lon_dir}'
        )

    def __eq__(self, other):
        """任何比较的魔术方式,都是两个参数,self代表自己
            可以使用==来比较
        Args:
            other (同self一个种类): 比较的对象
        return:
            只有所有属性都相等才能返回相等
        """
        if not isinstance(other, GlobalCoordinates):
            # 如果不是同一类,无法比较
            return NotImplemented
        return (
            self._lat_deg == other._lat_deg
            and self._lat_min == other._lat_min
            and self._lat_sec == other._lat_sec
            and self._lat_dir == other._lat_dir
            and self._lat_deg == other._lat_deg
            and self._lat_min == other._lat_min
            and self._lat_dir == other._lat_dir
        )

    def __sub__(self, other) -> tuple:
        if not isinstance(other, GlobalCoordinates):
            return NotImplemented
        lat_diff = self.latitude - other.latitude
        long_diff = self.longtitude - other.longtitude
        return (lat_diff, long_diff)

    def __invert__(self):
        return GlobalCoordinates(
            latitude=self.degrees_from_decimal(-self.latitude, lat=True),
            longitude=self.degrees_from_decimal(-self.longitude, lat=False),
        )
        """ 一些其他魔术方法
            __ne__() 表示不等,使用!=运算符
            __lt__() 表示小于 使用<运算符
            __gt__() 表示大于 使用>运算符
            __sub__() 表示相减 self-other
            __invert() 表示相反 使用~运算符
        """


nsp = GlobalCoordinates(latitude=(37, 46, 32.6, "N"), longtitude=(122, 24, 39.4, "W"))
print(nsp)  # 37°4632.6"N 122°2439.4"W  更加格式化好看

37°4632.6"N 122°2439.4"W


In [ ]:
# 类装饰器
class CoffeeOrder:
    def __init__(self, recipe, to_go=False):
        # ?? 什么意思
        self.recipe = recipe
        self.to_go = to_go  # 是否带走

    def brew(self):
        vessel = "in a paper cup." if self.to_go else "in a mug."
        print("Brewing", *self.recipe.parts, vessel)


class CoffeeRecipe:
    def __init__(self, parts):
        self.parts = parts


special = CoffeeRecipe(["double-shot", "grande", "no-whip", "mocha"])
order = CoffeeOrder(special, to_go=False)  # 传递进来一个类
order.brew()  # prints "Brewing double-shot grande no-whip mocha in a mug"